In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings("ignore")
os.mkdir('/kaggle/working/output_dir')

In [3]:
!pip install -U -q "huggingface_hub[cli]"

In [4]:
!huggingface-cli login --token [HF_Token]

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Data Preprocessing

In [5]:
from datasets import load_dataset

dataset = load_dataset("scitldr")

dataset['train'] = dataset['train'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

dataset['validation'] = dataset['validation'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

dataset['test'] = dataset['test'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

dataset

scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/7.56k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

The repository for scitldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scitldr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 1992
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 618
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 619
    })
})

In [6]:
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("gpt2")

def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["source"]])

# Map this function to dataset
tokenized_dataset = dataset.map(
                      preprocess_function,
                      batched= True,#Batch the data to increase speed
                      num_proc = 4, #Number of Processes | Parallelization step
                      remove_columns= dataset['train'].column_names
                      )

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/1992 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/618 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/619 [00:00<?, ? examples/s]

# SFT on Complete Model

In [8]:
from transformers import AutoConfig, AutoModelForCausalLM, TrainingArguments, Trainer
import math

# Import the model:
# Model_checkpoint = string containing model Id or Path to directory containing model weights save using save_pretrained()
# Kwargs generally mention change to configuration to the model while loading like loading a tensorflow model in pytorch using from_tf=True
model = AutoModelForCausalLM.from_pretrained('gpt2')

# Training the model:
# TrainingArguments can be found at https://huggingface.co/transformers/v4.7.0/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/RawSFT",
    eval_strategy="epoch",
    run_name="gpt2-summerizer",
    learning_rate=2e-5,
    weight_decay=0.01,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator, # For padding with EOS as mentioned earlier
    tokenizer=tokenizer,
)

trainer.train()

# Evaluation of results:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Epoch,Training Loss,Validation Loss
1,No log,3.419472
2,No log,3.375183
3,No log,3.364841


Perplexity: 28.93


In [ ]:
# To push the model to HuggingFace we can use:
trainer.create_model_card()
trainer.push_to_hub("HF_username/RawSFT")

# SFT LoRA

In [10]:
!pip install -q trl peft

In [11]:
from datasets import load_dataset
from trl import SFTTrainer 
from peft import LoraConfig
from random import randrange
from transformers import AutoConfig, AutoModelForCausalLM, TrainingArguments, Trainer 
import math 


# Create a data modification function like below if required to perform SFT.
# Here we are considering case of text-summarization as shown in blog: LoRA for Fine-Tuning LLMs explained with codes and example
# Change or remove this function based on your dataset.
def prompt_instruction_format(sample):
    return f"""### Instruction:
        Use the Task below and the Input given to write the Response:

        ### Task:
        Summarize the Input

        ### Input:
        {sample['source']}

        ### Response:
        {sample['target']}
        """ 

# Load your data that you want to fine-tune the existing model with
dataset = load_dataset("scitldr")

dataset['train'] = dataset['train'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

dataset['validation'] = dataset['validation'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

dataset['test'] = dataset['test'].remove_columns(['source_labels', 'rouge_scores', 'paper_id'])

# Load your pretrained model, either from huggingface or from your local
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Set new training arguments for fine-tuning 
trainingArgs = TrainingArguments(
    output_dir="/kaggle/working/SFT_with_LoRA",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_strategy="epoch",
    learning_rate=2e-4
)

# Create configuration for LoRA 
peft_config = LoraConfig(
      lora_alpha=16, # The alpha parameter for Lora scaling, adjusts the influence of LoRA adaptations, generally set 16, 32, or 64.
      lora_dropout=0.1, # Dropout Rate.
      r=64, #  Lora attention dimension (the “rank”).
      bias="none", # Bias type for LoRA. Can be ‘none’, ‘all’ or ‘lora_only’.
      task_type="CAUSAL_LM", # Type of task
      # target_modules = ["q_proj", "v_proj"] # Specify which layers of the model should be adapted by LoRA.
      # layers_to_transform: [0,1,2,4] # Control which layers are adapted by specifying their indices.
)

# Create a SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

# Evaluation of results:
eval_results = trainer.evaluate()
print(f"Perplexity Before Training: {math.exp(eval_results['eval_loss']):.2f}")

# Fine-tune the final model
trainer.train()

# Evaluation of results:
eval_results = trainer.evaluate()
print(f"Perplexity After Training: {math.exp(eval_results['eval_loss']):.2f}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Perplexity Before Training: 27.88


Step,Training Loss


Perplexity After Training: 11.57


In [ ]:
# To push the new model to HuggingFace we can use:
trainer.create_model_card()
trainer.push_to_hub("HF_username/SFT_with_LoRA")

# RLHF

### IMP NOTE: Restart the notebook, run the code before Data Preprocessing and then directly the cells below. This is for not getting any dependency or CUDA error

In [5]:
!pip install -q trl peft

In [7]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, TaskType, get_peft_model
from trl import RewardTrainer, RewardConfig

# Load the dataset and tokenizer
dataset = load_dataset("davanstrien/dataset-tldr-preference-dpo")
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Remove unnecessary columns from the dataset
dataset = dataset.remove_columns([
    'datasetId', 'card', 'generation_models', 
    'generations', 'model_name', 'ratings', 
    'rationales', 'prompt'
])

# Preprocessing function for tokenizing the "chosen" and "rejected" texts
def preprocess_function(examples):
    chosen = tokenizer([x for x in examples["chosen"]], padding="max_length", truncation=True)
    rejected = tokenizer([x for x in examples["rejected"]], padding="max_length", truncation=True)
    
    return {
        "input_ids_chosen": chosen["input_ids"],
        "attention_mask_chosen": chosen["attention_mask"],
        "input_ids_rejected": rejected["input_ids"],
        "attention_mask_rejected": rejected["attention_mask"]
    }

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,  # Process in batches to increase speed
    num_proc=4,    # Use parallel processing
    remove_columns=dataset['train'].column_names  # Remove unused columns after preprocessing
)

# Split the tokenized dataset into training and testing sets
tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)

# Ensure the padding token is set correctly
tokenizer.pad_token = tokenizer.eos_token

# Data collator to handle padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set model ID
model_id = "t5-small"

# Load the model and move it to the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.to(device)

# Align padding tokens between tokenizer and model
model.config.pad_token_id = tokenizer.pad_token_id

# Define PEFT (LoRA) configuration for efficient fine-tuning
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

# Define reward-specific training arguments
training_args = RewardConfig(
    output_dir="./output_dir/my_reward_model",  # Adjust the path for local storage
    eval_strategy="steps",
    center_rewards_coefficient=0.01,
    eval_steps=50,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8,  # Adjusted batch size
    max_length=50,
    do_eval=True,
)

# Define the RewardTrainer with TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    peft_config=peft_config
)

# Train the model
trainer.train()

# Save the model
trainer.save_model(training_args.output_dir)

# Evaluate and log metrics
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
50,No log,0.702054,0.333333


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ This dataset contains persona descriptions    │ The tl;dr summary of the dataset based on    │ [0.5153, 0.4847] │
│ and traits, which can be used to develop      │ the provided card is unavailable, as no      │                  │
│ chatbots or virtual assistants that can       │ dataset card has been supplied. Please       │                  │
│ engage in more personalized and human-like    │ provide a dataset card with information      │                  │
│ conversations.<pad><pad><pad><pad><pad><pad>… │ about a specific dataset to generate an      │                  │
│                                               │ accurate tl;dr                               │                  │
│                                               │ summary.<pad><pad><pad><pad><pad><pad><pad>… │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ The HelpSteer dataset contains 37,120 samples │ HelpSteer is a dataset containing 37,120     │ [0.5259, 0.4741] │
│ with prompts and responses, along with        │ samples of prompts, responses, and           │                  │
│ human-annotated attributes for helpfulness,   │ human-annotated attributes, enabling the     │                  │
│ correctness, coherence, complexity, and       │ training of models to become more helpful,   │                  │
│ verbosity, designed to aid in aligning models │ factually correct, coherent, and adjustable  │                  │
│ to provide more helpful, factually correct,   │ in terms of complexity and                   │                  │
│ and coherent                                  │ verbosity.<pad><pad><pad><pad><pad><pad><pa… │                  │
│ responses.<pad><pad><pad><pad><pad><pad><pad… │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ The Roboflow Website Screenshots dataset is a │ This dataset contains annotated screenshots  │ [0.5507, 0.4493] │
│ synthetic dataset containing over 1000        │ from over 1000 top websites, labeled with    │                  │
│ annotated screenshots from popular websites,  │ various HTML elements such as buttons,       │                  │
│ useful for Robotic Process Automation and     │ headings, links, and images, which can be    │                  │
│ offering a cost-effective alternative to      │ used for Robotic Process Automation and      │                  │
│ human                                         │ related                                      │                  │
│ labeling.<pad><pad><pad><pad><pad><pad><pad>… │ applications.<pad><pad><pad><pad><pad><pad>… │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ MultiCoNER is a fine-grained multilingual     │ The MultiCoNER dataset, available in 12      │ [0.5328, 0.4672] │
│ named entity recognition dataset covering 12  │ languages, is a fine-grained multilingual    │                  │
│ languages, including Bangla, Chinese,         │ named entity recognition (NER) resource      │                  │
│ English, and others, annotated with 11 entity │ designed for use in the SemEval-2023 Task    │                  │
│ types such as Location, Person, and Medical,  │ 2.<pad><pad><pad><pad><pad><pad><pad><pad><… │                  │
│ suitable for training and evaluating models   │                                              │                  │
│ for named entity recognition                  │       

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ This dataset contains persona descriptions    │ The tl;dr summary of the dataset based on    │ [0.4909, 0.5091] │
│ and traits, which can be used to develop      │ the provided card is unavailable, as no      │                  │
│ chatbots or virtual assistants that can       │ dataset card has been supplied. Please       │                  │
│ engage in more personalized and human-like    │ provide a dataset card with information      │                  │
│ conversations.<pad><pad><pad><pad><pad><pad>… │ about a specific dataset to generate an      │                  │
│                                               │ accurate tl;dr                               │                  │
│                                               │ summary.<pad><pad><pad><pad><pad><pad><pad>… │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ The HelpSteer dataset contains 37,120 samples │ HelpSteer is a dataset containing 37,120     │ [0.5879, 0.4121] │
│ with prompts and responses, along with        │ samples of prompts, responses, and           │                  │
│ human-annotated attributes for helpfulness,   │ human-annotated attributes, enabling the     │                  │
│ correctness, coherence, complexity, and       │ training of models to become more helpful,   │                  │
│ verbosity, designed to aid in aligning models │ factually correct, coherent, and adjustable  │                  │
│ to provide more helpful, factually correct,   │ in terms of complexity and                   │                  │
│ and coherent                                  │ verbosity.<pad><pad><pad><pad><pad><pad><pa… │                  │
│ responses.<pad><pad><pad><pad><pad><pad><pad… │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ The Roboflow Website Screenshots dataset is a │ This dataset contains annotated screenshots  │ [0.5049, 0.4951] │
│ synthetic dataset containing over 1000        │ from over 1000 top websites, labeled with    │                  │
│ annotated screenshots from popular websites,  │ various HTML elements such as buttons,       │                  │
│ useful for Robotic Process Automation and     │ headings, links, and images, which can be    │                  │
│ offering a cost-effective alternative to      │ used for Robotic Process Automation and      │                  │
│ human                                         │ related                                      │                  │
│ labeling.<pad><pad><pad><pad><pad><pad><pad>… │ applications.<pad><pad><pad><pad><pad><pad>… │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ MultiCoNER is a fine-grained multilingual     │ The MultiCoNER dataset, available in 12      │ [0.5486, 0.4514] │
│ named entity recognition dataset covering 12  │ languages, is a fine-grained multilingual    │                  │
│ languages, including Bangla, Chinese,         │ named entity recognition (NER) resource      │                  │
│ English, and others, annotated with 11 entity │ designed for use in the SemEval-2023 Task    │                  │
│ types such as Location, Person, and Medical,  │ 2.<pad><pad><pad><pad><pad><pad><pad><pad><… │                  │
│ suitable for training and evaluating models   │                                              │                  │
│ for named entity recognition                  │       

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.3333
  eval_loss               =      0.702
  eval_runtime            = 0:00:03.57
  eval_samples_per_second =     29.389
  eval_steps_per_second   =      1.959


In [8]:
# To push the new model to HuggingFace:
trainer.save_model("/kaggle/working/output_dir/my_reward_model")

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline, DataCollatorForLanguageModeling
from transformers import LogitsProcessorList, MinLengthLogitsProcessor
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("scitldr")

# Remove unwanted columns from all dataset splits
columns_to_remove = ['source_labels', 'rouge_scores', 'paper_id']
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].remove_columns(columns_to_remove)

# Set up PPO config with a GPT-like model (causal language model)
config = PPOConfig(
    model_name="gpt2",  # Use GPT-2 or another causal language model
    learning_rate=1.41e-5,
    mini_batch_size=1,
    batch_size=1,
    gradient_accumulation_steps=1
)

# Union function to concatenate the source field
def union(sample):
    return {'union': " ".join(sample['source']).strip()}

# Apply the union function to all dataset splits
union_dataset = dataset.map(
    union,
    batched=False,  # Process samples one at a time
    num_proc=4,     # Parallel processing for speed
    remove_columns=dataset['train'].column_names
)

# Load the GPT-2 model and tokenizer
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set correctly

# Set up reward model pipeline (use the model trained for rewards)
reward_model = pipeline(
    "summarization",
    model="/kaggle/working/output_dir/my_reward_model",  # Path to your reward model
    device=0 if torch.cuda.is_available() else -1
)

# Tokenization function
def tokenize(sample):
    return tokenizer(sample['union'], padding='max_length', max_length=200, truncation=True)

# Tokenize the dataset
tokenized_dataset = union_dataset.map(
    tokenize,
    batched=False,
    num_proc=4,
    remove_columns=union_dataset['train'].column_names
)

# Data collator to handle padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


eos_token_id = torch.tensor(tokenizer.eos_token_id).to(device)

logits_processor = LogitsProcessorList([
    MinLengthLogitsProcessor(min_length=10, eos_token_id=eos_token_id),  # Ensure min length
])

# Initialize PPOTrainer
ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=tokenized_dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Keywords to pass to model.generate at every SFT step
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_length": 400,
    "logits_processor":logits_processor
}

# Training loop
epochs = 1
for epoch in tqdm(range(epochs), desc="epoch"):
    for batch in tqdm(ppo_trainer.dataloader, desc="batch"): 
        query_tensors = batch["input_ids"]

        # Loop through the batch, generating responses for each sample individually
        for query_tensor in query_tensors:
            # Ensure query tensors are correctly shaped (1D tensors)
            query_tensor = query_tensor.squeeze()

            print(tokenizer.decode(query_tensor))

            # Generate responses using the model (wrap query_tensor in a list)
            response_tensors = ppo_trainer.generate([query_tensor], **generation_kwargs)
            batch["target"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

            # Combine queries and responses for reward calculation
            texts = [tokenizer.decode(query_tensor) + tokenizer.decode(r.squeeze()) for r in response_tensors]
            
            # Get the summaries from the reward model (summarization)
            summaries = reward_model(texts)
            print(summaries)

            # Calculate a simple reward based on length difference (convert rewards to float)
            rewards = []
            for query, summary in zip(tokenizer.decode(query_tensor), summaries):
                # A simple reward calculation: length difference between original and summary
                reward = torch.tensor(len(summary["summary_text"]) - len(query)).float()  # Cast to float
                rewards.append(reward)

            # Perform PPO step (pass query_tensor and response_tensors as lists)
            stats = ppo_trainer.step([query_tensor], response_tensors, rewards)
            ppo_trainer.log_stats(stats, batch, rewards)

# Save the PPO-trained model
ppo_trainer.save_model("/kaggle/working/output_dir/my_ppo_model")

# Evaluate the results
eval_results = ppo_trainer.evaluate()
print(f"Perplexity: {torch.exp(torch.tensor(eval_results['eval_loss'])):.2f}")


batch:   0%|          | 0/1992 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In this paper, we explore new approaches to combining information encoded within the learned representations of autoencoders. We explore models that are capable of combining the attributes of multiple inputs such that a resynthesised output is trained to fool an adversarial discriminator for real versus synthesised data. Furthermore, we explore the use of such an architecture in the context of semi-supervised learning, where we learn a mixing function whose objective is to produce interpolations of hidden states, or masked combinations of latent representations that are consistent with a conditioned class label. We show quantitative and qualitative evidence that such a formulation is an interesting avenue of research.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

Token indices sequence length is longer than the specified maximum sequence length for this model (812 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'in this paper, we explore new approaches to combining information encoded within the learned representations of autoencoders . we explore models that are capable of combining attributes of multiple inputs such that a resynthesised output is trained to fool an adversarial discriminator . also explore the use of such an architecture in the context of semi-supervised learning .'}]



batch:   0%|          | 1/1992 [00:03<2:12:28,  3.99s/it]

...

Disentangling underlying generative factors of a data distribution is important for interpretability and generalizable representations. In this paper,  we introduce two novel disentangling methods. Our first method, Unlabeled Disentangling GAN (UD-GAN, unsupervised), decomposes the latent noise by generating similar/dissimilar image pairs and it learns a distance metric on these pairs with siamese networks and a contrastive loss. This pairwise approach provides consistent representations for similar data points. Our second method (UD-GAN-G, weakly supervised) modifies the UD-GAN with user-defined guidance functions, which restrict the information that goes into the siamese networks. This constraint helps UD-GAN-G to focus on the desired semantic variations in the data. We  show  that  both  our  methods  outperform  existing  unsupervised approaches in quantitative metrics that measure semantic accuracy of the learned representations. In addition, we illustrate that
[{'summary_text': '


batch:  11%|█▏        | 225/1992 [14:23<1:53:03,  3.84s/it]

What can we learn about the functional organization of cortical microcircuits from large-scale recordings of neural activity?   To obtain an explicit and interpretable model of time-dependent functional connections between neurons and to establish the dynamics of the cortical information flow, we develop 'dynamic neural relational inference' (dNRI). We study  both synthetic and real-world neural spiking data and demonstrate that the developed method is able to uncover the dynamic relations between neurons more reliably than existing baselines.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endofte


batch:  11%|█▏        | 226/1992 [14:27<1:50:16,  3.75s/it]

We consider two questions at the heart of machine learning; how can we predict if a minimum will generalize to the test set, and why does stochastic gradient descent find minima that generalize well? Our work responds to \citet{zhang2016understanding}, who showed deep neural networks can easily memorize randomly labeled training data, despite generalizing well on real labels of the same inputs. We show that the same phenomenon occurs in small linear models. These observations are explained by the Bayesian evidence, which penalizes sharp minima but is invariant to model parameterization. We also demonstrate that, when one holds the learning rate fixed, there is an optimum batch size which maximizes the test set accuracy. We propose that the noise introduced by small mini-batches drives the parameters towards minima whose evidence is large. Interpreting stochastic gradient descent as a stochastic differential equation, we identify the ``noise scale" $g = \epsilon
